In [1]:
import pandas as pd
import os, fnmatch
import numpy as np
from scipy.stats import iqr
from numpy import percentile
from nipype import Node, Workflow, MapNode
# it's important to
# pass filenames to Nodes as absolute paths
from os.path import abspath
from nipype.interfaces import afni, fsl
from nipype.workflows.fmri.fsl.preprocess import create_susan_smooth 
import nibabel as nib
#for getting the results of 1dBport into a data frame
import subprocess
from io import StringIO

def locate(pattern, root=os.curdir):
    '''Locate all files matching supplied filename pattern in and below
    supplied root directory.'''
    for path, dirs, files in os.walk(os.path.abspath(root)):
        for filename in fnmatch.filter(files, pattern):
            return os.path.join(path, filename)
            #yield os.path.join(path, filename)
def create_motion_reg_matrix(confounds_filename, motion_reg_colnames, spikereg_colnames, use_derivs, use_quads, dvar_reg, displace_reg, displace_thresh):
    confounds = pd.read_csv(confounds_filename, sep = '\t').reindex(columns = motion_reg_colnames + spikereg_colnames)
    confounds_motion_reg = confounds[motion_reg_colnames]
    confounds_spikereg = confounds[spikereg_colnames]

    if use_derivs == True:
        deriv = np.diff(confounds_motion_reg, n=1, axis=0)
        deriv_df = pd.DataFrame(np.insert(arr=deriv, obj=0, values=0, axis=0), 
                                columns=[s + '_deriv' for s in confounds_motion_reg.columns])
        confounds_motion_all = pd.concat([confounds_motion_reg, deriv_df], axis=1)
    else:
        confounds_motion_all = confounds_motion_reg

    if use_quads == True:
        quads_df = confounds_motion_all.apply(np.square)
        quads_df.columns = [s + '_sq' for s in quads_df.columns]
        confounds_motion_all = pd.concat([confounds_motion_all, quads_df], axis=1)

    #Compute some more columns for the confounds df
    #from https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLMotionOutliers:
    #--thresh=<val>       specify absolute threshold value (otherwise use box-plot cutoff = P75 + 1.5*IQR)
    spikes = pd.Series([0]*confounds_spikereg.shape[0])

    if dvar_reg == False and displace_reg == False:
        raise Exception('Must include at least one spike regressor option.')

    #Mark volumes that meet outlier criteria
    if dvar_reg == True:
        dvar_outlier_boundry = percentile(confounds_spikereg.loc[1:, 'dvars'], q = [75]) + 1.5*iqr(confounds_spikereg.loc[1:, 'dvars'])
        spikes = spikes + confounds_spikereg['dvars'].apply(lambda dvar: 1 if dvar > dvar_outlier_boundry else 0)
    if displace_reg == True:
        spikes = spikes + confounds_spikereg['framewise_displacement'].apply(lambda fwdisp: 1 if fwdisp > displace_thresh else 0)
    #because of the addition above, make sure we only have 0 or 1 in this series.
    spikes = spikes.apply(lambda spike: 1 if spike > 0 else 0)

    #Create spike regressors:
    spikes_df = spikes.to_frame('spike')
    spikes_df['tr'] = pd.Series(range(1, spikes_df.shape[0] + 1)).astype(str).str.zfill(3)
    spikes_df_l = spikes_df.melt(id_vars = 'tr')
    spikes_df_l['spike_reg'] = spikes_df_l['variable'] + spikes_df_l['tr']
    spike_reg_w = spikes_df_l[spikes_df_l['value'] == 1].pivot(index='tr', columns='spike_reg', values='value')
    spike_reg_w_trs = spikes_df.join(spike_reg_w, on = 'tr', how = 'left').fillna(0)

    #concatenate continuous and spike regressors
    all_motion_confound_regressors = pd.concat([confounds_motion_all, spike_reg_w_trs.drop(['spike', 'tr'], axis = 1)], axis = 1)

    #make sure we don't have too many regressors
    if(all_motion_confound_regressors.shape[0] < all_motion_confound_regressors.shape[1]):
        raise Exception('Too many regressors (' + str(all_motion_confound_regressors.shape[1]) + ') for number of TRs (' + str(all_motion_confound_regressors.shape[0]) + ').')

    return(all_motion_confound_regressors)            

In [2]:
root = '/net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/'

In [11]:
found_confounds = locate('sub-*confounds_regressors.tsv', root)
found_rsbold = locate('sub-*MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', root)
found_mask = locate('*desc-brain_mask.nii.gz', root)

In [12]:
found_mask

'/net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/sub-1001/ses-1/func/sub-1001_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'

In [4]:
confounds_filename = found_confounds
#18 regressors are these 9, and their derivatives.
motion_reg_colnames = ['csf', 'white_matter', 'global_signal', 'trans_x', 'trans_y', 'trans_z', 'rot_x' , 'rot_y', 'rot_z']
#To compute spike regressors
spikereg_colnames = ['dvars', 'framewise_displacement']
use_derivs = True
use_quads = False
dvar_reg = True
displace_reg = True
displace_thresh = 0.1 #mm
confound_regressors = create_motion_reg_matrix(confounds_filename = confounds_filename, motion_reg_colnames = motion_reg_colnames,
                        spikereg_colnames = spikereg_colnames, use_derivs = use_derivs,
                        use_quads = use_quads, dvar_reg = dvar_reg, displace_reg = displace_reg,
                        displace_thresh = displace_thresh)

In [5]:
in_file = abspath(found_rsbold)
number_trs = nib.load(in_file).shape[3]

In [6]:
#create bandpass regressors
result = subprocess.run(['1dBport', '-nodata', str(number_trs), '2', '-band', '0.01', '.08', '-invert', '-nozero'], stdout=subprocess.PIPE)

In [7]:
bandpass_string = StringIO(result.stdout.decode('utf-8'))
bandpass_df = pd.read_csv(bandpass_string, sep='\s+', header=None)

In [8]:
bandpass_df

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,1.000000,0.000000,1.000000,0.000000,1.000000e+00,0.000000,1.000000,0.000000e+00,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,-1
1,0.999391,0.034899,0.997564,0.069756,9.945220e-01,0.104528,0.500000,8.660250e-01,0.469472,0.882948,...,0.173648,-0.990268,0.139173,-0.994522,0.104528,-0.997564,0.069756,-0.999391,0.034899,1
2,0.997564,0.069756,0.990268,0.139173,9.781480e-01,0.207912,-0.500000,8.660250e-01,-0.559193,0.829037,...,-0.342019,0.961262,-0.275637,0.978148,-0.207911,0.990268,-0.139173,0.997564,-0.069756,-1
3,0.994522,0.104528,0.978148,0.207912,9.510560e-01,0.309017,-1.000000,-8.742280e-08,-0.994522,-0.104529,...,0.499999,-0.913546,0.406735,-0.951057,0.309015,-0.978148,0.207911,-0.994522,0.104528,1
4,0.990268,0.139173,0.961262,0.275637,9.135450e-01,0.406737,-0.500000,-8.660250e-01,-0.374606,-0.927184,...,-0.642787,0.848049,-0.529918,0.913546,-0.406735,0.961262,-0.275636,0.990268,-0.139172,-1
5,0.984808,0.173648,0.939693,0.342020,8.660250e-01,0.500000,0.500000,-8.660250e-01,0.642788,-0.766044,...,0.766043,-0.766046,0.642786,-0.866026,0.499999,-0.939693,0.342019,-0.984808,0.173647,1
6,0.978148,0.207912,0.913545,0.406737,8.090170e-01,0.587785,1.000000,1.748460e-07,0.978148,0.207912,...,-0.866024,0.669133,-0.743143,0.809019,-0.587782,0.913546,-0.406734,0.978148,-0.207910,-1
7,0.970296,0.241922,0.882948,0.469472,7.431450e-01,0.669131,0.500000,8.660260e-01,0.275637,0.961262,...,0.939692,-0.559195,0.829036,-0.743147,0.669128,-0.882949,0.469470,-0.970296,0.241919,1
8,0.961262,0.275637,0.848048,0.529919,6.691310e-01,0.743145,-0.500000,8.660250e-01,-0.719340,0.694658,...,-0.984807,0.438374,-0.898793,0.669133,-0.743142,0.848049,-0.529918,0.961262,-0.275635,-1
9,0.951056,0.309017,0.809017,0.587785,5.877850e-01,0.809017,-1.000000,-2.384980e-08,-0.951056,-0.309017,...,1.000000,-0.309020,0.951055,-0.587788,0.809015,-0.809018,0.587784,-0.951057,0.309015,1


In [9]:
despike_out_file = abspath(root + '/tmp/despike.nii.gz')

In [10]:
workflow = Workflow(name = 'motion_correct')
#set up the 3dDespike node, which will be connected to the susan, 3dDeconvolve, and 3dTproject nodes
despike = Node(afni.Despike(in_file = in_file, out_file = despike_out_file,
                            args = '-overwrite -NEW -nomask'), name = "despike")

despike.base_dir = abspath(root + '/tmp')

In [17]:
smooth = create_susan_smooth()
smooth.inputs.inputnode.fwhm = 5
smooth.inputs.inputnode.in_files = despike_out_file
smooth.inputs.inputnode.mask_file = abspath(found_mask)
smooth.run()

190417-17:52:00,569 nipype.workflow INFO:
	 Workflow susan_smooth settings: ['check', 'execution', 'logging', 'monitoring']
190417-17:52:00,625 nipype.workflow INFO:
	 Running serially.
190417-17:52:00,628 nipype.workflow INFO:
	 [Node] Setting-up "susan_smooth.mask" in "/tmp/tmp7hvtvgpm/susan_smooth/mask".
190417-17:52:00,644 nipype.workflow INFO:
	 [Node] Setting-up "_mask0" in "/tmp/tmp7hvtvgpm/susan_smooth/mask/mapflow/_mask0".
190417-17:52:00,654 nipype.workflow INFO:
	 [Node] Running "_mask0" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/tmp/despike.nii.gz -mas /net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/sub-1001/ses-1/func/sub-1001_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz /tmp/tmp7hvtvgpm/susan_smooth/mask/mapflow/_mask0/despike_mask.nii.gz
190417-17:52:00,743 nipype.interface INFO:



190417-17:52:01,128 nipype.workflow INFO:
	 [Node] Setting-up "susan_smooth.median" in "/tmp/tmpxa0i9co6/susan_smooth/median".
190417-17:52:01,142 nipype.workflow INFO:
	 [Node] Setting-up "_median0" in "/tmp/tmpxa0i9co6/susan_smooth/median/mapflow/_median0".
190417-17:52:01,152 nipype.workflow INFO:
	 [Node] Running "_median0" ("nipype.interfaces.fsl.utils.ImageStats"), a CommandLine Interface with command:
fslstats /net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/tmp/despike.nii.gz -k /net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/sub-1001/ses-1/func/sub-1001_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz -p 50 
190417-17:52:01,242 nipype.interface INFO:
	 stderr 2019-04-17T17:52:01.241891:/bin/sh: port_used: line 1: syntax error: unexpected end of file
190417-17:52:01,246 nipype.interface INFO:
	 stderr 2019-04-17T17:52:01.241891:/bin/sh: error importing function definition for `BASH_FUN

190417-17:52:01,492 nipype.workflow ERROR:
	 could not run node: susan_smooth.median
190417-17:52:01,494 nipype.workflow INFO:
	 crashfile: /net/holynfs01/srv/export/mclaughlin/share_root/users/jflournoy/code/GenR/post_processing/crash-20190417-175201-jflournoy-median-de694cdb-8ea0-48d8-94ea-e1e16b19e6ab.pklz
190417-17:52:01,497 nipype.workflow INFO:
	 ***********************************


RuntimeError: Workflow did not execute cleanly. Check log for details

In [20]:
abspath(found_mask)

'/net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/sub-1001/ses-1/func/sub-1001_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'